IMPORTANT: Make sure to run this notebook first (Create Mapping File from Echonest Mapping Project) in order to create 'mapping.csv' (if not already available)

Import the Dataset and have an initial exploration, see shifts and possible issues.

In [1]:
import h5py
import pandas as pd
from glob import glob, iglob
import json

In [2]:
%cd ../data/raw/
%ls

/Users/sebastian/git_repos/data_science/hit_predictor/data/raw
MillionSongFullSummary/ MillionSongSubset/      faq/


First we'll look at the summary file

In [3]:
data = h5py.File('MillionSongFullSummary/msd_summary_file.h5','r')
for key in data.keys():
    print(key)

analysis
metadata
musicbrainz


In [4]:
for key in data['musicbrainz'].keys():
    print(key)
    
analysis_songs = pd.DataFrame(data['analysis']['songs'].value)
analysis_songs.head()

songs


/Users/sebastian/anaconda3/envs/hit_predictor/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,...,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id
0,22050,b'aee9820911781c734e7694c5432990ca',0.0,252.05506,2.049,0.0,0,0,0,0,...,10,0.777,-4.829,0,0.688,236.635,87.002,4,0.940,b'TRMMMYQ128F932D901'
1,22050,b'ed222d07c83bac7689d52753610a513a',0.0,156.55138,0.258,0.0,0,0,0,0,...,9,0.808,-10.555,1,0.355,148.660,150.778,1,0.000,b'TRMMMKD128F425225D'
2,22050,b'96c7104889a128fef84fa469d60e380c',0.0,138.97098,0.000,0.0,0,0,0,0,...,7,0.418,-2.060,1,0.566,138.971,177.768,4,0.446,b'TRMMMRX128F93187D9'
3,22050,b'0f7da84b6b583e3846c7e022fb3a92a2',0.0,145.05751,0.000,0.0,0,0,0,0,...,7,0.125,-4.654,1,0.451,138.687,87.433,4,0.000,b'TRMMMCH128F425532C'
4,22050,b'228dd6392ad8001b0281f533f34c72fd',0.0,514.29832,0.000,0.0,0,0,0,0,...,5,0.097,-7.806,0,0.290,506.717,140.035,4,0.315,b'TRMMMWA128F426B589'


In [14]:
analysis_songs.head(1)['track_id'].str.decode('utf-8')[0]

b'TRMMMYQ128F932D901'


'TRMMMYQ128F932D901'

In [48]:
# Load Metadata
meta_data_songs = pd.DataFrame(data['metadata']['songs'].value)

/Users/sebastian/anaconda3/envs/hit_predictor/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [49]:
meta_reduced = meta_data_songs.loc[:,['artist_name','artist_mbid','title','song_id']].copy()

In [47]:
# Transform bytes to strings
meta_reduced.loc[~meta_reduced['artist_mbid'].isnull(),'artist_mbid'] = meta_reduced.loc[~meta_reduced['artist_mbid'].isnull(),'artist_mbid'].str.decode('utf-8')
meta_reduced.loc[~meta_reduced['artist_name'].isnull(),'artist_name'] = meta_reduced.loc[~meta_reduced['artist_name'].isnull(),'artist_name'].str.decode('utf-8')
meta_reduced.loc[~meta_reduced['song_id'].isnull(),'song_id'] = meta_reduced.loc[~meta_reduced['song_id'].isnull(),'song_id'].str.decode('utf-8')
meta_reduced.loc[~meta_reduced['title'].isnull(),'title'] = meta_reduced.loc[~meta_reduced['title'].isnull(),'title'].str.decode('utf-8')

# Test output
meta_reduced.head()

,artist_name,artist_mbid,title,song_id
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [45]:
# Import mapping.csv file (see notebook Create Mapping File from Echonest Mapping Project)
mapping_df = pd.read_csv('../interim/mapping_summary.csv',sep='\t').drop_duplicates(subset='id',keep='first')
mapping_df.head()

,tracks.catalog,tracks.foreign_id,tracks.id,artist_name,id,title
0,spotify,spotify:track:4UvblPNa9nwMTkzLtUBxnj,TRNAQME144D1525B11,Jackson Sisters,SOCSZMF12AAF3B3498,Why Do Fools Fall In Love
1,spotify,spotify:track:2biysHaswSmcpAM1T554BR,TROAEGA144D0FE9F1A,Journey,SOCSMOE12A8C1459E8,Live and Breathe
3,spotify,spotify:track:4YkMlmjR7KVwbZa3D3ek8N,TRYQDDR144D154494E,Chamillionaire,SOCBVBO1315CD46DFA,Think I'm Crazy
12,spotify,spotify:track:7oZHYB2HXjvgcMfu8tTqBA,TRCSAFR147B6EB3D98,Moonspell,SOCSZXL12A6D4F8145,Opium
17,spotify,spotify:track:3CSkXJgKEupR2mYx6CnW8c,TRJDLNF144D118CFAF,Oliver Huntemann,SOCSBRR12A6D4FBEC2,37 Grad (Arnaud Rebotini's Blackstrobe Remix)


In [46]:
meta_reduced_sp_ids = meta_reduced.merge(mapping_df,left_on=['song_id','artist_name','title'],right_on=['id','artist_name','title'],how='left')
meta_reduced_sp_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
artist_name          1000000 non-null object
artist_mbid          1000000 non-null object
title                1000000 non-null object
song_id              1000000 non-null object
tracks.catalog       286028 non-null object
tracks.foreign_id    286028 non-null object
tracks.id            286028 non-null object
id                   286028 non-null object
dtypes: object(8)
memory usage: 68.7+ MB
